# Notebook 2 — Modèles de bases (classification, clustering, réduction de dimension)

**Objectifs :**
- Utiliser **Régression Logistique** (classification)
- Utiliser **k-NN** (classification)
- Utiliser **k-means** (clustering)
- Utiliser **ACP / PCA** (réduction de dimension)
- Évaluer les modèles et diagnostiquer **underfitting / overfitting**

> Ce notebook est volontairement **guidé** : certaines lignes sont à compléter (TODO) pour vous amener à raisonner.

---
## Consignes
- Lisez les cellules markdown (explications) puis complétez les cellules de code **aux endroits marqués `TODO`**.
- Tant que vous n'avez pas complété, certaines cellules **sautent** l'exécution pour éviter les erreurs.
- Travaillez proprement : variables claires, commentaires, et interprétation des résultats.


In [5]:
# --- Imports & config ---
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt

np.random.seed(42)


## 1) Dataset de classification (binaire) : Breast Cancer Wisconsin
On va utiliser un dataset classique (déjà inclus dans scikit-learn) pour entraîner des modèles de **classification**.

**Pourquoi ce dataset ?**
- Beaucoup d'exemples
- Variables numériques (facile à normaliser)
- Deux classes (bénin / malin) → parfait pour la régression logistique et k-NN


In [6]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")  # 0/1

print("Shape X:", X.shape)
print("Classes y:", y.value_counts().to_dict())
X.head()


Shape X: (569, 30)
Classes y: {1: 357, 0: 212}


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### 1.1 Split train / test
On sépare le dataset en **train** et **test** pour estimer la performance sur des données non vues.

À compléter :
- `test_size` (ex : 0.2)
- `random_state` (ex : 42)
- `stratify=y` (important en classification)


In [7]:
# TODO: complétez les paramètres pour un split reproductible et stratifié
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=...,          # TODO
    random_state=...,       # TODO
    stratify=...            # TODO
)

print("Train:", X_train.shape, " Test:", X_test.shape)


InvalidParameterError: The 'test_size' parameter of train_test_split must be a float in the range (0.0, 1.0), an int in the range [1, inf) or None. Got Ellipsis instead.

## 2) Régression Logistique
La **régression logistique** est un modèle linéaire pour la classification. On l'utilise souvent comme baseline.

### 2.1 Pipeline (StandardScaler + LogisticRegression)
On met un `StandardScaler` avant, car la régularisation et la descente d'optimisation dépendent de l'échelle des features.

**À faire :**
- Importer `LogisticRegression`
- Créer un pipeline `scaler + modèle`
- Entraîner (`fit`) puis prédire (`predict`) et évaluer (`accuracy`, matrice de confusion)


In [ ]:
from sklearn.linear_model import LogisticRegression

# TODO: choisissez une valeur de C (ex: 1.0). Plus C est grand, moins il y a de régularisation.
C_value = ...  # TODO

log_reg_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(
        C=C_value,
        max_iter=500,
        # TODO: si besoin, précisez solver=...
    ))
])

# --- Exécution protégée : tant que C_value n'est pas rempli, on saute ---
if C_value is ...:
    print(" Complétez C_value (TODO) pour entraîner la régression logistique.")
else:
    log_reg_pipe.fit(X_train, y_train)
    y_pred = log_reg_pipe.predict(X_test)

    print("Accuracy test:", accuracy_score(y_test, y_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nRapport:\n", classification_report(y_test, y_pred, target_names=data.target_names))


### 2.2 Overfitting / Underfitting via validation curve (paramètre C)
Idée : comparer la performance **train** vs **validation** en faisant varier un hyperparamètre.

- Si **train élevé** et **val faible** → *overfitting*
- Si **train faible** et **val faible** → *underfitting*

On va tracer une **validation curve** sur `C`.

**À compléter :**
- La liste de valeurs `C_grid`
- L'appel à `validation_curve`


In [ ]:
from sklearn.model_selection import validation_curve

# TODO: grille de C (ex: np.logspace(-3, 3, 7))
C_grid = ...  # TODO

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=500))
])

if C_grid is ...:
    print(" Complétez C_grid (TODO) pour tracer la validation curve.")
else:
    train_scores, val_scores = validation_curve(
        pipe,
        X_train, y_train,
        param_name="model__C",
        param_range=C_grid,
        cv=5,
        scoring="accuracy"
    )

    train_mean = train_scores.mean(axis=1)
    val_mean = val_scores.mean(axis=1)

    plt.figure()
    plt.semilogx(C_grid, train_mean, marker="o", label="train")
    plt.semilogx(C_grid, val_mean, marker="o", label="validation")
    plt.xlabel("C (log scale)")
    plt.ylabel("accuracy")
    plt.title("Validation curve — Logistic Regression (C)")
    plt.legend()
    plt.show()

    print("→ Interprétez la zone où l'écart train/val est grand (overfitting) et où les deux sont faibles (underfitting).")


## 3) k-NN (k plus proches voisins)
k-NN prédit la classe d'un point en regardant les **k voisins** les plus proches (distance).

**Point clé :** k-NN est très sensible à l'échelle des variables → on normalise.

### 3.1 Entraîner et évaluer un k-NN
**À compléter :**
- Importer `KNeighborsClassifier`
- Choisir une valeur `k`
- Entraîner et évaluer sur test


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

k = ...  # TODO: par ex 3, 5, 11...

knn_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier(n_neighbors=k))
])

if k is ...:
    print(" Complétez k (TODO) pour entraîner k-NN.")
else:
    knn_pipe.fit(X_train, y_train)
    y_pred = knn_pipe.predict(X_test)
    print("Accuracy test:", accuracy_score(y_test, y_pred))


### 3.2 Diagnostiquer under/overfitting en faisant varier k
Intuition :
- **k petit** → modèle très flexible (risque d'overfitting)
- **k grand** → modèle plus lisse (risque d'underfitting)

**À compléter :**
- Une grille `k_values`
- Calculer accuracy sur train et test (ou validation)
- Tracer les courbes


In [ ]:
k_values = ...  # TODO: ex: range(1, 31, 2)

if k_values is ...:
    print(" Complétez k_values (TODO).")
else:
    train_acc = []
    test_acc = []

    for k in k_values:
        pipe = Pipeline([
            ("scaler", StandardScaler()),
            ("model", KNeighborsClassifier(n_neighbors=k))
        ])
        pipe.fit(X_train, y_train)

        # TODO: calculez les accuracies train et test
        y_pred_train = ...  # TODO
        y_pred_test = ...   # TODO

        train_acc.append(accuracy_score(y_train, y_pred_train))
        test_acc.append(accuracy_score(y_test, y_pred_test))

    plt.figure()
    plt.plot(list(k_values), train_acc, marker="o", label="train")
    plt.plot(list(k_values), test_acc, marker="o", label="test")
    plt.xlabel("k (n_neighbors)")
    plt.ylabel("accuracy")
    plt.title("Under/Overfitting — k-NN")
    plt.legend()
    plt.show()

    print("→ Où est-ce que le modèle overfit ? Où est-ce qu'il underfit ?")


## 4) Clustering : k-means (non supervisé)
Ici, pas de labels `y` : on cherche à regrouper les points en **k clusters**.

On va utiliser le dataset **Iris** (3 espèces) uniquement pour visualiser : k-means n'utilise PAS les espèces.


In [ ]:
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans

iris = load_iris()
X_iris = pd.DataFrame(iris.data, columns=iris.feature_names)

X_iris.head()


### 4.1 Choisir k : méthode du coude (inertia)
k-means minimise la somme des distances intra-cluster (inertia).
On trace inertia en fonction de k et on cherche un 'coude'.

**À compléter :**
- Une grille de `k` (ex: 1..10)
- Calculer `inertia_` pour chaque k


In [ ]:
k_grid = ...  # TODO: ex range(1, 11)

if k_grid is ...:
    print(" Complétez k_grid (TODO).")
else:
    inertia = []
    for k in k_grid:
        km = KMeans(n_clusters=k, n_init=10, random_state=42)
        km.fit(X_iris)
        inertia.append(km.inertia_)

    plt.figure()
    plt.plot(list(k_grid), inertia, marker="o")
    plt.xlabel("k")
    plt.ylabel("inertia")
    plt.title("Méthode du coude — k-means")
    plt.show()

    print("→ Choisissez un k raisonnable d'après le coude.")


### 4.2 Qualité de clustering : silhouette score
Le **silhouette score** (entre -1 et 1) évalue si les clusters sont bien séparés.

**À compléter :**
- Importer `silhouette_score`
- Calculer le score pour plusieurs k (k>=2)


In [ ]:
from sklearn.metrics import silhouette_score

k_grid2 = ...  # TODO: ex range(2, 11)

if k_grid2 is ...:
    print(" Complétez k_grid2 (TODO).")
else:
    scores = []
    for k in k_grid2:
        km = KMeans(n_clusters=k, n_init=10, random_state=42)
        labels = km.fit_predict(X_iris)
        scores.append(silhouette_score(X_iris, labels))

    plt.figure()
    plt.plot(list(k_grid2), scores, marker="o")
    plt.xlabel("k")
    plt.ylabel("silhouette score")
    plt.title("Silhouette score — k-means")
    plt.show()

    best_k = list(k_grid2)[int(np.argmax(scores))]
    print("Meilleur k (selon silhouette):", best_k)


## 5) Réduction de dimension : ACP / PCA
L'ACP (PCA) projette les données dans un espace de dimension plus faible en conservant un maximum de variance.

**À quoi ça sert ?**
- Visualisation (2D)
- Réduction du bruit
- Accélérer certains algorithmes

On va appliquer PCA sur Iris, après standardisation.


In [ ]:
from sklearn.decomposition import PCA

# Standardisation + PCA
pca = PCA(n_components=2, random_state=42)

pipe_pca = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", pca)
])

X_2d = pipe_pca.fit_transform(X_iris)

print("X_2d shape:", X_2d.shape)
print("Variance expliquée (2 comps):", pca.explained_variance_ratio_.sum())


### 5.1 Visualisation 2D (PCA)
On colorie par les vraies espèces *uniquement pour comprendre* ; PCA n'utilise pas ces labels.


In [ ]:
plt.figure()
plt.scatter(X_2d[:, 0], X_2d[:, 1], c=iris.target)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Iris projeté en 2D via PCA")
plt.show()


### 5.2 k-means après PCA
On peut faire k-means sur les composantes PCA (2D) :
- plus rapide
- parfois plus séparé

**À compléter :**
- Choisir `k`
- Entraîner k-means sur `X_2d`
- Visualiser les clusters


In [ ]:
k = ...  # TODO: essayez 2, 3, 4...

if k is ...:
    print(" Complétez k (TODO) pour k-means sur PCA.")
else:
    km = KMeans(n_clusters=k, n_init=20, random_state=42)
    labels = km.fit_predict(X_2d)

    plt.figure()
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c=labels)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(f"k-means (k={k}) sur projection PCA 2D")
    plt.show()


## 6)  Cross-validation sur les modèles supervisés
Au lieu d'un seul split train/test, on estime la performance avec une **cross-validation**.

**À compléter :**
- Calculer `cross_val_score` pour la régression logistique et pour k-NN
- Comparer moyenne et écart-type


In [ ]:
# TODO: choisissez un nombre de folds (cv)
cv = ...  # TODO: ex 5

if cv is ...:
    print(" Complétez cv (TODO).")
else:
    # Logistic regression
    log_reg = Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=500))
    ])
    scores_lr = cross_val_score(log_reg, X, y, cv=cv, scoring="accuracy")

    # k-NN (choisissez k)
    k = ...  # TODO
    if k is ...:
        print(" Complétez k (TODO) pour la CV de k-NN.")
    else:
        knn = Pipeline([
            ("scaler", StandardScaler()),
            ("model", KNeighborsClassifier(n_neighbors=k))
        ])
        scores_knn = cross_val_score(knn, X, y, cv=cv, scoring="accuracy")

        print(f"LogReg CV accuracy: {scores_lr.mean():.3f} ± {scores_lr.std():.3f}")
        print(f"kNN(k={k}) CV accuracy: {scores_knn.mean():.3f} ± {scores_knn.std():.3f}")


---
## À rendre / Questions
1. Régression logistique : quelle valeur de **C** donne le meilleur compromis train/val ? Pourquoi ?
2. k-NN : pour quelles valeurs de **k** observe-t-on overfitting / underfitting ?
3. k-means : quel k choisissez-vous (coude vs silhouette) ? Comparez.
4. PCA : quelle proportion de variance est expliquée par 2 composantes ? Est-ce suffisant ?
5. (Bonus) Comparez les performances CV de LogReg vs k-NN.
